In [ ]:
%matplotlib inline

%pylab inline
pylab.rcParams['figure.figsize'] = (12,10)

import os
import cv2
import matplotlib.pyplot as plt
import numpy as np

#data_path = '/Users/ryoungblood/opencv_tests/data/'
data_path = '/Users/ryoungblood/image_similarity/original_images/'

In [ ]:
boot = cv2.imread(os.path.join(data_path,'15.jpg'))
plt.imshow(cv2.cvtColor(boot,cv2.COLOR_BGR2RGB))

In [ ]:
#cv2.imwrite(os.path.join(data_path,'boot2.png'),boot)

In [ ]:
boot_gray = cv2.cvtColor(boot, cv2.COLOR_BGR2GRAY)
plt.imshow(boot_gray, cmap='gray')

In [ ]:
boot_negate = 255 - boot_gray
plt.imshow(boot_negate, cmap='gray')

In [ ]:
# Get the horizontal Sobel
boot_sobel_h_64f = cv2.Sobel(boot_negate,cv2.CV_64F,1,0,ksize=3)
boot_sobel_h_64f_abs = np.absolute(boot_sobel_h_64f)
boot_sobel_h_8u = np.uint8(boot_sobel_h_64f_abs)

# Get the vertical Sobel
boot_sobel_v_64f = cv2.Sobel(boot_negate,cv2.CV_64F,0,1,ksize=3)
boot_sobel_v_64f_abs = np.absolute(boot_sobel_v_64f)
boot_sobel_v_8u = np.uint8(boot_sobel_v_64f_abs)

# Bitwise OR those two
boot_sobel = cv2.bitwise_or(boot_sobel_h_8u,boot_sobel_v_8u)
print(boot_sobel[:10,:10])

plt.imshow(boot_sobel, cmap='gray')

In [ ]:
boot_gauss = cv2.GaussianBlur(boot_sobel, (9,9), 0)
plt.imshow(boot_gauss, cmap='gray')

In [ ]:
print(boot_gauss.shape)
print(boot_gauss[:10,:10])

In [ ]:
threshold = 40

ret,boot_binary = cv2.threshold(boot_gauss,threshold,255,cv2.THRESH_BINARY)
ret,boot_binary_inv = cv2.threshold(boot_gauss,threshold,255,cv2.THRESH_BINARY_INV)
ret,boot_trunc = cv2.threshold(boot_gauss,threshold,255,cv2.THRESH_TRUNC)
ret,boot_tozero = cv2.threshold(boot_gauss,threshold,255,cv2.THRESH_TOZERO)
ret,boot_tozero_inv = cv2.threshold(boot_gauss,threshold,255,cv2.THRESH_TOZERO_INV)

titles = ['Original Image','BINARY','BINARY_INV','TRUNC','TOZERO','TOZERO_INV']
images = [boot_gauss, boot_binary, boot_binary_inv, boot_trunc, 
          boot_tozero, boot_tozero_inv]

for i in range(6):
    plt.subplot(2,3,i+1),plt.imshow(images[i],'gray')
    plt.title(titles[i])
    plt.xticks([]),plt.yticks([])

plt.show()

#plt.imshow(binary, cmap='gray')


In [ ]:
print('TOZERO')
plt.imshow(boot_tozero, cmap='gray')

In [ ]:
boot_tozero_bgr = cv2.cvtColor(boot_tozero, cv2.COLOR_GRAY2BGR)
plt.imshow(boot_tozero_bgr)

In [ ]:
mask = np.zeros((boot_tozero.shape[0]+2,boot_tozero.shape[1]+2),np.uint8)
diff = (6,6,6)
cv2.floodFill(image=boot_tozero_bgr,mask=mask,seedPoint=(0,0),
             newVal=(255,0,255),loDiff=diff,upDiff=diff, flags=4)
cv2.floodFill(image=boot_tozero_bgr,mask=mask,seedPoint=(boot_tozero.shape[1]-1,0),
             newVal=(255,0,255),loDiff=diff,upDiff=diff, flags=4)
cv2.floodFill(image=boot_tozero_bgr,mask=mask,seedPoint=(boot_tozero.shape[1]-1,boot_tozero.shape[0]-1),
             newVal=(255,0,255),loDiff=diff,upDiff=diff, flags=4)

cv2.floodFill(image=boot_tozero_bgr,mask=mask,seedPoint=(0,boot_tozero.shape[0]-1),
             newVal=(255,0,255),loDiff=diff,upDiff=diff, flags=4)
plt.imshow(boot_tozero_bgr)

In [ ]:
plt.imshow(mask, cmap='gray')

In [ ]:
mask = mask * 255

In [ ]:
mask_bgr = cv2.cvtColor(mask[1:-1,1:-1],cv2.COLOR_GRAY2BGR)

In [ ]:
boot_bgremoved = cv2.bitwise_or(mask_bgr,boot)
plt.imshow(cv2.cvtColor(boot_bgremoved,cv2.COLOR_BGR2RGB))